In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
RE = "Solar_PBE" # Solar_PBE Wind_Wallonie_Elia

address = "https://raw.githubusercontent.com/Jaeik-Jeong/DeepBid/main/results/"

data_test_csv   = pd.read_csv(address+RE+'_19.csv', index_col=0)
data_price = pd.read_csv(address+'Price_Elia_Imbalance_19.csv', index_col=0)
data_test_csv['Price(€)']  = data_price['Positive imbalance price']

In [ ]:
# Data Preprocessing

RE_Capacity3 = max(data_test_csv['Power(MW)'])
max_price = max(data_price['Marginal incremental price'])

unit    = 1 #unit: 15 minute
E_max   = 0.15 #p.u.
P_max   = E_max
tdelta  = unit/4
soc_min = 0.1
soc_max = 0.9
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288
N = 130*215*E_max/0.1
beta = 10/max_price

size_test0  = int(len(data_test_csv)/unit)
data_test0 = []; data_test = []; price_test0 = []; price_test = []; time_test = []
for i in range(size_test0):
    data_test0  += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    price_test0 += [round(pd.Series.mean(data_test_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_test0[i] > 0: data_test += [data_test0[i]]; price_test += [price_test0[i]]; time_test += [data_test_csv.index[i]]

In [ ]:
metric = ['Profit', 'Penalty', 'Battery_Use', 'Revenue']
 
profit  = []
penalty = []
bat_use = []
revenue = []
Bid = []
SoC = []

start_time = 0
end_time  = 32
 
results = {}
def function(x):
    select_test = x[:int(len(x)/2)]
    select_test_rat = x[int(len(x)/2):]
    select_test_real = np.array(data_test[1:][start_time:end_time])
    select_test_price = np.array(price_test[1:][start_time:end_time])
    
    E = E_max/2
    revenue = 0
    for i in range(end_time - start_time):
        bid = select_test[i]
        gen = select_test_real[i]
        rat = select_test_rat[i]
        imb = select_test_price[i]
        
        soc = E/E_max
        Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
        Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
        Rts = c0*np.exp(-c1*soc) + c2
        Rtl = d0*np.exp(-d1*soc) + d2
        R   = Rs + Rts + Rtl
    
        I_cmax = 1000000*E_max*(soc_max - soc)/N/(Voc*tdelta)
        I_dmax = 1000000*E_max*(soc - soc_min)/N/(Voc*tdelta)
        p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
        p_dmax = N*(Voc*I_dmax - I_dmax**2*R)
    
        P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
        P_c = min(max(rat*(gen-bid), 0), P_max, P_cmax)
        P_d = min(max(rat*(bid-gen), 0), P_max, P_dmax)
        p_c = 1000000*P_c/N; p_d = 1000000*P_d/N
    
        I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
        I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
        if not np.isclose(p_c, 0):
            eff_c = (Voc*I_c)/p_c
            E = E + eff_c*P_c*tdelta
            disp = gen - P_c
        elif not np.isclose(p_d, 0):
            eff_d = p_d/(Voc*I_d)
            E = E - (1/eff_d)*P_d*tdelta
            disp = gen + P_d
        else:
            disp = gen
        
        revenue += (imb*disp - imb*abs(bid-disp) - beta*(P_c+P_d))*tdelta
    
    return -revenue

In [ ]:
num_scenario = 50
for i in range(num_scenario):
    Init_Point = np.array(data_test[1:][start_time:end_time]+price_test[1:][start_time:end_time])
    Optimum = minimize(function, Init_Point, method='CG', options={'maxiter':100, 'disp':True})
    try:
        Optimum_x += Optimum.x
    except:
        Optimum_x = Optimum.x

Optimum_x = Optimum_x/50